Procedemos a realizar el script del alquiler

In [1]:
import sqlite3
import pandas as pd
import numpy as np

# Cargar el archivo CSV
ruta_csv = 'alquiler_inmuebles_limpio.csv'
datos = pd.read_csv(ruta_csv)

# Limpiar y transformar datos
# Reemplazar valores "No especificado" y NaN con None para insertar en la base de datos
datos.replace("No especificado", np.nan, inplace=True)
datos.fillna(value=np.nan, inplace=True)  # Reemplaza NaN con None al insertarlo en SQL

# Procesar columnas específicas
# Extraer solo los números en "Consumo Energético" y "Emisiones Co2"
datos['consumo_energetico'] = datos['Consumo Energético'].str.extract(r'(\d+)').astype(float)
datos['emisiones_co2'] = datos['Emisiones Co2'].str.extract(r'(\d+)').astype(float)

# Conectar a la base de datos (se creará si no existe)
conn = sqlite3.connect('alquiler_madrid.db')
cursor = conn.cursor()

# Crear la tabla en SQLite
cursor.execute('''
    CREATE TABLE IF NOT EXISTS inmuebles (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        descripcion TEXT,
        localizacion TEXT,
        precio REAL,
        ultima_actualizacion DATE,
        consumo_energetico REAL,
        emisiones_co2 REAL,
        tipo_operacion TEXT,
        superficie_construida REAL,
        superficie_util REAL,
        habitaciones INTEGER,
        baños INTEGER,
        antigüedad TEXT,
        conservacion TEXT,
        planta TEXT,
        tipo_casa TEXT
    )
''')

# Preparar los datos para inserción en la tabla
for _, fila in datos.iterrows():
    cursor.execute('''
        INSERT INTO inmuebles (descripcion, localizacion, precio, ultima_actualizacion,
                               consumo_energetico, emisiones_co2, tipo_operacion, 
                               superficie_construida, superficie_util, habitaciones, 
                               baños, antigüedad, conservacion, planta, tipo_casa)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        fila['Descripción'], 
        fila['Localización'], 
        fila['Precio'], 
        fila['Última Actualización'], 
        fila['consumo_energetico'], 
        fila['emisiones_co2'], 
        fila['Tipo De Operación'], 
        fila['Superficie Construida'], 
        float(fila['Superficie Útil']) if pd.notnull(fila['Superficie Útil']) else None, 
        int(fila['Habitaciones']) if pd.notnull(fila['Habitaciones']) else None, 
        int(fila['Baños']) if pd.notnull(fila['Baños']) else None, 
        fila['Antigüedad'], 
        fila['Conservación'], 
        fila['Planta'], 
        fila['Tipo De Casa']
    ))

# Guardar los cambios
conn.commit()

# Verificar los datos insertados
cursor.execute("SELECT * FROM inmuebles LIMIT 5")
resultados = cursor.fetchall()
for fila in resultados:
    print(fila)

# Cerrar la conexión
conn.close()


(1, 'Casa unifamiliar en alquiler en Calle XIX', 'Las Matas-Los Peñascales (Las Rozas de Madrid)', 4000.0, '2024-10-28', None, None, 'Alquiler', 600.0, '600.0', '6', '7', 'Entre 5 y 10 años', 'En buen estado', 'No especificado', 'Casa')
(2, 'Piso en alquiler en Zona Avenida de Europa', 'Zona Avenida de Europa (Pozuelo de Alarcón)', 1400.0, '2024-10-28', None, None, 'Alquiler', 65.0, 'No especificado', '1', '1', 'No especificado', 'No especificado', '2ª', 'Piso')
(3, 'Piso en alquiler en Calle Luis Garcia Cereceda, Número 3', 'La Finca (Pozuelo de Alarcón)', 5500.0, '2024-10-28', 'Consumo:15 kWh/m² año', 'Emisiones:2 Kg CO₂/m² año', 'Alquiler', 190.0, '170.0', '3', '3', 'Menos de 5 años', 'En buen estado', '2ª', 'Piso')
(4, 'Estudio en alquiler en Calle de Pollensa, 5', 'Parque Empresarial (Las Rozas de Madrid)', 1147.0, '2024-07-03', None, None, 'Alquiler', 50.0, 'No especificado', 'No especificado', '1', 'No especificado', 'No especificado', 'No especificado', 'Estudio')
(5, 'Piso en 

Ahora procedemos a realizar el script de las ventas ,inclyendo la  creanción de la tabla en el mismo código

In [1]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime
import re

# Ruta del archivo CSV y el archivo de base de datos
csv_path = 'venta_inmuebles_limpio.csv'
db_path = 'inmuebles_venta.db'

# Cargar el archivo CSV en un DataFrame de pandas
datos = pd.read_csv(csv_path)

# Limpieza y transformación de datos
datos.replace("No especificado", np.nan, inplace=True)
datos['Consumo Energético'] = datos['Consumo Energético'].astype(float)
datos['Emisiones CO2'] = datos['Emisiones CO2'].astype(float)

# Convertir la fecha de 'Última Actualización' al formato DATE de SQLite
def convertir_fecha(fecha):
    try:
        return datetime.strptime(fecha, '%d/%m/%Y').date() if pd.notnull(fecha) else None
    except ValueError:
        return None

datos['Última Actualización'] = datos['Última Actualización'].apply(convertir_fecha)

# Función para limpiar y convertir "Gastos de comunidad" a un valor numérico
def limpiar_gastos(gasto):
    if pd.isnull(gasto):
        return None
    numeros = re.findall(r'\d+', str(gasto))
    if len(numeros) == 1:
        return float(numeros[0])
    elif len(numeros) > 1:
        return (float(numeros[0]) + float(numeros[1])) / 2
    else:
        return None

datos['Gastos de comunidad:'] = datos['Gastos de comunidad:'].apply(limpiar_gastos)

# Conectar a la base de datos con un tiempo de espera de 10 segundos
conn = sqlite3.connect(db_path, timeout=10)
cursor = conn.cursor()

# Crear la tabla `inmuebles_venta`
create_table_query = """
CREATE TABLE IF NOT EXISTS inmuebles_venta (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    descripcion TEXT,
    localizacion TEXT,
    precio REAL,
    ultima_actualizacion DATE,
    consumo_energetico REAL,
    emisiones_co2 REAL,
    tipo_operacion TEXT,
    superficie_construida REAL,
    superficie_util REAL,
    habitaciones INTEGER,
    baños INTEGER,
    antigüedad TEXT,
    conservacion TEXT,
    planta TEXT,
    tipo_casa TEXT,
    gastos_comunidad REAL,
    exterior TEXT,
    interior TEXT,
    referencia TEXT
);
"""
cursor.execute(create_table_query)

# Insertar los datos en la tabla `inmuebles_venta`
for _, fila in datos.iterrows():
    cursor.execute('''
        INSERT INTO inmuebles_venta (descripcion, localizacion, precio, ultima_actualizacion,
                                     consumo_energetico, emisiones_co2, tipo_operacion, 
                                     superficie_construida, superficie_util, habitaciones, 
                                     baños, antigüedad, conservacion, planta, tipo_casa,
                                     gastos_comunidad, exterior, interior, referencia)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        fila['descripcion'], 
        fila['Enlace'], 
        float(fila['Precio']) if pd.notnull(fila['Precio']) else None, 
        fila['Última Actualización'], 
        fila['Consumo Energético'], 
        fila['Emisiones CO2'], 
        fila['Tipo de operación'], 
        float(fila['Superficie construida:']) if pd.notnull(fila['Superficie construida:']) else None, 
        float(fila['Superficie útil:']) if pd.notnull(fila['Superficie útil:']) else None, 
        int(fila['Habitaciones:']) if pd.notnull(fila['Habitaciones:']) else None, 
        int(fila['Baños:']) if pd.notnull(fila['Baños:']) else None, 
        fila['Antigüedad:'], 
        fila['Conservación:'], 
        fila['Planta:'], 
        fila['Tipo de casa:'], 
        fila['Gastos de comunidad:'],  
        fila['Exterior:'], 
        fila['Interior:'], 
        fila['Referencia:']
    ))

# Confirmar cambios y cerrar la conexión
conn.commit()
conn.close()
print("Datos insertados exitosamente en `inmuebles_venta` desde el archivo CSV.")


C:\Users\guill\AppData\Local\Temp\ipykernel_19672\3873441834.py:75: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute('''


Datos insertados exitosamente en `inmuebles_venta` desde el archivo CSV.
